# Where could you build ADUs in Waltham before the rezoning?


## The most optimistic case - any SFH plot can build an ADU

This isn't exactly practical for all plots (there is a minimum size of plot or ADU which would make construction unviable) but just as a quick check of the absolute maximum potential:

In [ ]:
# TODO: get tax parcels, filter to SFH use codes and count

: 

## Diving into the regulations

The section of our zoning code describing the regulations for ADUs can be found at the link below

https://ecode360.com/13128595#13128572

Already in the first part of the first full sentence we're off to a bad start - the Board of Appeals must grant a special permit to allow an ADU. No way for there to be a by-right development by outlining
a clear set of rules and ministerial approval that the rules were followed to the letter. This is discouraging enough on its own.

## (a)

_The proposed dwelling unit is in harmony with and will promote the purposes of this chapter._

This clause seems oddly worded to me. Do they mean 'the regulations below do not override other regulations in this chapter?' This is too vague to check so I won't be doing it here.

## (b)

_The dwelling unit is accessory to the principal residence and either the dwelling unit or the principal residence is occupied by the owner of the lot on which the dwelling unit is to be located, which owner shall have owned the lot for not less than five years._

This is the kind of owner occupancy clause that the state law is specifically forbidding. It's just nonsense - why must the owner reside on the lot? And why a five year minimum residency requirement?

We don't have a foolproof way of checking residency, so we'll go with our current method of 'matching a lot of characters' in the owner address on the tax records matching the site address.

In [ ]:
# TODO: get ownership table, find SFH parcels

## Requirement for pre-existing buildings